<a href="https://colab.research.google.com/github/gunwooCho2/11-11-TeamPractice/blob/main/hello_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GPU 기반의 고성능 CUDA 프로그래밍**
## 한림대학교 소프트웨어 융합대학 이정근
## 2019년 2월

안녕하세요. 반갑습니다. 한림대학교 소프트웨어융합대학 이정근 교수라고 합니다.
이번 GPU CUDA 교육에 참여해 주셔서 감사합니다. 본 강의는 이론과 실습으로 이루어져 있으며, 실습은 Colab을 통해서 진행될 예정입니다.

우선 교육에 참여하신 모든 선생님들께서는 Github 및 Colab 계정을 만들어 주시면 감사드리겠습니다.

* Github 주소: www.github.com
* Colab 주소: colab.research.google.com
* Google Drive 연동 : https://github.com/jeonggunlee/CUDATeaching/blob/master/colab_gdrive.ipynb

*  *  *

Colab에서 CUDA Coding을 실습하기 위해서는 몇가지 사항을 알아야합니다.

* [코드 셀]에서 command-line 명령어 실행시키기
   - !ls : 현재 디렉토리의 내용을 보여준다.
   - %cd *dir*: *dir* 디렉토리로 이동한다.
   - %pwd: 현재 위치한 디렉토리 위치를 보여준다.
   - !git: git 명령어를 실행시킨다.

* 또한 GPU 장치와 관련되어 다음 명령어를 확인해주시기 바랍니다.
   - !nvidia-smi: 현재 사용하고 있는 GPU의 스펙과 작동 상황을 보여준다.
   - !nvcc: Nvidia CUDA Compiler를 실행시킨다.
   
위의 명령어들을 실행 시켜 보시기 바랍니다.

아래의 내용은 youtube 동영상을 통하여 확인할 수 있습니다. [colab을 이용한 GPU CUDA 프로그래밍 ](https://youtu.be/pT38R3jXwe0)


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import h5py
import os

# ✅ 1. U-Net 모델 정의
def build_unet_model(input_shape=(None, None, 3)):
    inputs = layers.Input(shape=input_shape)

    # 인코더
    c1 = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D()(c1)

    c2 = layers.Conv2D(64, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D()(c2)

    # 병목
    b = layers.Conv2D(128, 3, activation='relu', padding='same')(p2)
    b = layers.Conv2D(128, 3, activation='relu', padding='same')(b)

    # 디코더
    u1 = layers.Conv2DTranspose(64, 2, strides=2, padding='same')(b)
    u1 = layers.concatenate([u1, c2])
    c3 = layers.Conv2D(64, 3, activation='relu', padding='same')(u1)
    c3 = layers.Conv2D(64, 3, activation='relu', padding='same')(c3)

    u2 = layers.Conv2DTranspose(32, 2, strides=2, padding='same')(c3)
    u2 = layers.concatenate([u2, c1])
    c4 = layers.Conv2D(32, 3, activation='relu', padding='same')(u2)
    c4 = layers.Conv2D(32, 3, activation='relu', padding='same')(c4)

    outputs = layers.Conv2D(3, 1, activation='sigmoid')(c4)

    return models.Model(inputs, outputs)

# ✅ 2. Dataset 클래스 정의
class HDF5Image2ImageDataset(tf.keras.utils.Sequence):
    def __init__(self, h5_path, batch_size=16, shuffle=True):
        self.h5 = h5py.File(h5_path, 'r')
        self.inputs = self.h5['korean_image']
        self.labels = self.h5['korean_label']
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.inputs))

    def __len__(self):
        return len(self.inputs) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_inputs = np.array([self.inputs[i] for i in batch_indices], dtype=np.float32) / 255.0
        batch_labels = np.array([self.labels[i] for i in batch_indices], dtype=np.float32) / 255.0

        return batch_inputs, batch_labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# ✅ 3. 학습 함수 정의
def train_model(h5_path, epochs=10, batch_size=16):
    dataset = HDF5Image2ImageDataset(h5_path, batch_size=batch_size)
    model = build_unet_model(input_shape=(None, None, 3))

    model.compile(
        optimizer='adam',
        loss='mae',  # 또는 'mse'
        metrics=['mae']
    )

    model.fit(dataset, epochs=epochs)

    # ✅ 모델 저장
    model.save("/content/drive/MyDrive/model/unet_image2image_model.h5")
    print("✅ 모델 저장 완료: unet_image2image_model.h5")

# ✅ 4. 실행 블록
if __name__ == '__main__':
    h5_path = "/content/drive/MyDrive/dataset/word_image_train_label.h5"

    if not os.path.exists(h5_path):
        raise FileNotFoundError(f"HDF5 파일이 존재하지 않습니다: {h5_path}")

    train_model(h5_path, epochs=10, batch_size=16)


FileNotFoundError: HDF5 파일이 존재하지 않습니다: content/User_data/word_image_train_label.h5

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


처음 명령어로 pwd 명령어를 실행시켜보도록 하겠습니다. pwd는 사용자가 위치한 디렉토리를 보여줍니다.

In [ ]:
!pwd

/content


현재의 디렉토리가 content에 위치하고 있네요. 기본적으로 colab을 사용하는 사용자는 항상 위와 같은 위치를 초기 디렉토리로 사용하게 됩니다.
다음으로 Unix/Linux 계열의 가장 기본적인 명령인 ls를 사용해보도록 하겠습니다. ls 명령어는 현재 디렉토리에 있는 화일 또는 디렉토리를 보여줍니다.

In [ ]:
!ls

sample_data


위에서 보는 바와 같이 sample_data라는 디렉토리가 하위 디렉토리에 있다는 것을 알 수 있습니다.

다음으로 우리가 사용하고 있는 시스템의 프로세서 및 GPU에 대해서 알아보도록 하지요.
우선 CPU에 대한 정보를 얻기 위해서 Proc file 시스템의 cpuinfo를 cat 명령어로 보도록 해보지요.

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs itlb_multihit
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management

메모리 및 디스크 사용량을 알아볼까요 ?

1.   메모리 사용량: !cat /proc/meminfo
2.   디스크 사용량: !df -h



In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          69G   32G   34G  49% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.8G     0  5.8G   0% /dev/shm
tmpfs           6.4G   16K  6.4G   1% /var/colab
/dev/sda1        75G   33G   43G  44% /opt/bin
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


자 다음엔 GPU의 스펙을 살펴보도록 하지요. GPU에 대한 정보를 얻기 위해서는 nvidia-smi 명령어를 사용합니다.

In [ ]:
!nvidia-smi

Sat May 30 15:56:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

CPU와 GPU에 대한 정보를 확인 한 후에는 우리가 수행하고자 하는 코드들을 github로 부터 가져와서 시행해보도록 하겠습니다.

저같은 경우는 https://github.com/jeonggunlee/CUDATeaching 의 내용을 git 명령어를 통하여 clone 해오도록 하겠습니다.


In [ ]:
!git clone https://github.com/jeonggunlee/CUDATeaching

Cloning into 'CUDATeaching'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 564 (delta 15), reused 0 (delta 0), pack-reused 536
Receiving objects: 100% (564/564), 31.11 MiB | 39.63 MiB/s, done.
Resolving deltas: 100% (308/308), done.


Git clone이 완료된 후에 올바로 clone이 되었는지 확인하기 위하여 ls 명령어를 수행해보독 하겠습니다.

In [ ]:
!ls

CUDATeaching  sample_data


이후, cd 명령어를 이용하여 CUDATeaching 디렉토리로 들어가보도록 하겠습니다. 주의 할점은 시스템 명령어를 시행할때 ! 또는 %를 붙여주어야 한다는 것입니다.

In [ ]:
%cd CUDATeaching

/content/CUDATeaching


다시 CUDATeaching 디렉토리로 들어온 후에 ls 명령어를 통하여 디렉토리에 어떤 화일이 있는지 살펴보도록 하겠습니다.

In [ ]:
!ls

00_googleDrive_CUDAExam.ipynb	03_cuda_lab		  images
01_cuda_lab			03_numba_vectorize.ipynb  PPTs
01_PyCUDA_simple_example.ipynb	colab_gdrive.ipynb	  README.md
02_cuda_lab			hello_CUDA.ipynb


위 화일 및 디렉토리 중,01_cuda_lab에 들어가 보도록 하겠습니다. 01_cuda_lab에는 실습을 위한 cuda 소스 코드들이 있습니다.

In [ ]:
%cd 01_cuda_lab

/content/CUDATeaching/01_cuda_lab


01_cuda_lab에 들어온후에 ls 명령어를 통하여 어떤 화일들이 있는지 확인해보록 하겠습니다.

In [ ]:
!ls

01_simple.ipynb      05_vectorAdd.ipynb		     09_coalMemory.ipynb
02_openmp.ipynb      06_2DIndex.ipynb		     clock.cu
03_simple_avx.ipynb  07_memoryType.ipynb	     README.md
04_helloCUDA.ipynb   08_DeviceQuery_Bandwidth.ipynb


가장 기본 적인 코드로 1hello.cu 를 컴파일 하고 실행시켜보도록 하겠습니다.
1hello.cu의 내용은 아래와 같습니다.

```cuda
#include <stdio.h>

__global__ void helloCUDA(void)
{
  printf("Hello thread %d in block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{
  helloCUDA<<<3, 4>>>();
  cudaDeviceReset();
  return 0;
}
```


In [ ]:
%%writefile 1hello.cu

# include <stdio.h>

__global__ void helloCUDA(void)
{
  printf("Hello thread %d in block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{
  helloCUDA<<<3, 4>>>();
  cudaDeviceReset();
  return 0;
}

Writing 1hello.cu


1hello.cu를 컴파일하기 위하여 nvdia GPU 컴파일러 명령어인 nvcc를 사용합니다.
nvcc 컴파일러가 어디에 위치하는지 먼저 살펴볼까요 ?

In [ ]:
!which nvcc

/usr/local/cuda/bin/nvcc


위의 명령어를 통하여 nvcc 컴파일러가 /usr/local/cuda/bin/에 위치하고 있음을 알 수 있습니다.
다음은 nvidia GPU 카드가 설치 되었는지 확인해보는 작업을 진행해보겠습니다.


In [ ]:
!ls -l /dev/nv*

crw-rw-rw- 1 root root 195,   0 May 30 15:54 /dev/nvidia0
crw-rw-rw- 1 root root 195, 255 May 30 15:54 /dev/nvidiactl
crw-rw-rw- 1 root root 247,   0 May 30 15:54 /dev/nvidia-uvm
crw-rw-rw- 1 root root 247,   1 May 30 15:54 /dev/nvidia-uvm-tools


위의 nvidia0 및 nvidiactrl 등을 통하여 nvidia GPU 장치가 설치되어 있음을 알 수 있습니다.

자 그럼 장치와 컴파일러가 잘 설치되어 있음을 확인하였으니, nvcc 명령어를 이용하여 CUDA 코드를 컴파일을 해보도록 하겠습니다.

명령어의 가장 단순한 컴파일 형식은 다음과 같습니다.
   * nvcc cuda_code.cu -o executable_output
   * 실예] nvcc ./1hello.cu -o 1hello
   

In [ ]:
!nvcc ./1hello.cu

이후, 컴파일된 실행화일인 a.out을 실행시키면 해당 결과가 프린트 됩니다.

In [ ]:
!./a.out

Hello thread 0 in block 0
Hello thread 1 in block 0
Hello thread 2 in block 0
Hello thread 3 in block 0
Hello thread 0 in block 1
Hello thread 1 in block 1
Hello thread 2 in block 1
Hello thread 3 in block 1
Hello thread 0 in block 2
Hello thread 1 in block 2
Hello thread 2 in block 2
Hello thread 3 in block 2


위의 코드를 수정하고 싶다면 어떻게 하면 좋을까요 ?
" %%writefile hello_cuda.cu "를 이용해서 진행하면 좋을 것 같습니다.


In [ ]:
%%writefile hello_cuda.cu
# include <stdio.h>

__global__ void helloCUDA(void)
{
  printf("Hello thread %d in block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{
  helloCUDA<<<2, 3>>>();
  cudaDeviceReset();
  return 0;
}


Overwriting hello_cuda.cu


In [ ]:
!nvcc ./hello_cuda.cu -o hello_cuda

In [ ]:
!ls

01_simple.ipynb      06_2DIndex.ipynb		     hello_cuda
02_openmp.ipynb      07_memoryType.ipynb	     hello_cuda.cu
03_simple_avx.ipynb  08_DeviceQuery_Bandwidth.ipynb  README.md
04_helloCUDA.ipynb   09_coalMemory.ipynb
05_vectorAdd.ipynb   clock.cu


In [ ]:
!./a.out

Hello thread 0 in block 0
Hello thread 1 in block 0
Hello thread 2 in block 0
Hello thread 3 in block 0
Hello thread 0 in block 1
Hello thread 1 in block 1
Hello thread 2 in block 1
Hello thread 3 in block 1
Hello thread 0 in block 2
Hello thread 1 in block 2
Hello thread 2 in block 2
Hello thread 3 in block 2


오늘 colaboratory를 활용하여 간단히 GPU CUDA Programming을 하는 방법을 보여드렸습니다. 잘 활용하시면 좋을 것 같네요!
감사합니다.
